In [13]:
import findspark
findspark.init()
from pyspark.sql import SparkSession

In [5]:
spark = SparkSession.builder.master("local[*]").appName('Practica_pyspark').getOrCreate()

In [38]:
# 1. Crear un DataFrame con datos de personas, que incluya nombre, edad y ciudad de residencia.
data = [("Alice", 25, "New York"),
        ("Bob", 30, "Los Angeles"),
        ("Charlie", 22, "Chicago")]

columns = ['nombre', 'edad', 'ciudad']

df = spark.createDataFrame(data, columns)

In [24]:
# 2. Mostrar solo los nombres de las personas del DataFrame.
df.select("nombre").show()

+-------+
| nombre|
+-------+
|  Alice|
|    Bob|
|Charlie|
+-------+



In [12]:
# 3. Filtrar personas cuya edad sea mayor o igual a 25.
df.filter(df["edad"] >= 25).show()

+------+----+-----------+
|nombre|edad|     ciudad|
+------+----+-----------+
| Alice|  25|   New York|
|   Bob|  30|Los Angeles|
+------+----+-----------+



In [25]:
# 4. Agregar una nueva columna "Pais" con un valor constante para todas las filas.
from pyspark.sql.functions import lit

df = df.withColumn("pais", lit("Estados Unidos"))

In [26]:
# 5. Calcular el promedio de edad de todas las personas.
from pyspark.sql.functions import avg

df.select(avg("edad")).show()

+------------------+
|         avg(edad)|
+------------------+
|25.666666666666668|
+------------------+



In [16]:
# 6. Ordenar el DataFrame por edad en orden descendente.
df.orderBy(df["edad"].desc()).show()

+-------+----+-----------+--------------+
| nombre|edad|     ciudad|          pais|
+-------+----+-----------+--------------+
|    Bob|  30|Los Angeles|Estados Unidos|
|  Alice|  25|   New York|Estados Unidos|
|Charlie|  22|    Chicago|Estados Unidos|
+-------+----+-----------+--------------+



In [18]:
# 7. Agrupar por ciudad y calcular la cantidad de personas en cada ciudad.
df.groupBy("ciudad").count().show()

+-----------+-----+
|     ciudad|count|
+-----------+-----+
|   New York|    1|
|Los Angeles|    1|
|    Chicago|    1|
+-----------+-----+



In [44]:
# 8. Renombrar la columna "Nombre" a "NombreCompleto".
df = df.withColumnRenamed("nombre", "nombreCompleto")

In [36]:
# 9. Eliminar la columna "Edad" del DataFrame.
df_drop = df.drop("edad")

In [39]:
# 10. Realizar una consulta SQL en el DataFrame para seleccionar personas mayores de 20 años.
df.createOrReplaceTempView("personas")
resultado_sql = spark.sql("SELECT * FROM personas WHERE edad >= 20")
resultado_sql.show()

+-------+----+-----------+
| nombre|edad|     ciudad|
+-------+----+-----------+
|  Alice|  25|   New York|
|    Bob|  30|Los Angeles|
|Charlie|  22|    Chicago|
+-------+----+-----------+



In [29]:
# 11. Calcular la suma total de todas las edades.
from pyspark.sql.functions import sum

df.select(sum("edad")).show()

+---------+
|sum(edad)|
+---------+
|       77|
+---------+



In [42]:
# 12. Calcular la edad mínima y máxima de todas las personas.
from pyspark.sql.functions import min, max

df.select(min("Edad"), max("edad")).show()

+---------+---------+
|min(Edad)|max(edad)|
+---------+---------+
|       22|       30|
+---------+---------+



In [43]:
# 13. Filtrar personas cuya ciudad de residencia sea "Chicago" y edad sea menor de 30.
df.filter((df["ciudad"] == "Chicago") & (df["edad"] < 30)).show()

+-------+----+-------+---------+
| nombre|edad| ciudad|RangoEdad|
+-------+----+-------+---------+
|Charlie|  22|Chicago|    21-40|
+-------+----+-------+---------+



In [32]:
# 14. Agregar una nueva columna "EdadDuplicada" que contenga el doble de la edad.
df = df.withColumn("edadDuplicada", df["edad"] * 2)

In [33]:
# 15. Convertir todas las edades en años a meses.
df = df.withColumn("edadEnMeses", df["edad"] * 12)

In [34]:
# 16. Contar el número total de personas en el DataFrame.
total_personas = df.count()
print("Total de personas:", total_personas)

Total de personas: 3


In [35]:
# 17. Filtrar personas cuya edad sea un número par.
df.filter(df["edad"] % 2 == 0).show()

+-------+----+-----------+--------------+-------------+-----------+
| nombre|edad|     ciudad|          pais|edadDuplicada|edadEnMeses|
+-------+----+-----------+--------------+-------------+-----------+
|    Bob|  30|Los Angeles|Estados Unidos|           60|        360|
|Charlie|  22|    Chicago|Estados Unidos|           44|        264|
+-------+----+-----------+--------------+-------------+-----------+



In [40]:
# 18. Calcular la cantidad de personas por rango de edades (0-20, 21-40, 41-60, 61+).
from pyspark.sql.functions import when

df = df.withColumn("RangoEdad", when((df["edad"] >= 0) & (df["edad"] <= 20), "0-20")
                    .when((df["edad"] >= 21) & (df["edad"] <= 40), "21-40")
                    .when((df["edad"] >= 41) & (df["edad"] <= 60), "41-60")
                    .otherwise("61+"))

df.groupBy("RangoEdad").count().show()

+---------+-----+
|RangoEdad|count|
+---------+-----+
|    21-40|    3|
+---------+-----+



In [45]:
# 19. Contar cuántas personas tienen el mismo nombre.
df.groupBy("nombreCompleto").count().show()

+--------------+-----+
|nombreCompleto|count|
+--------------+-----+
|         Alice|    1|
|           Bob|    1|
|       Charlie|    1|
+--------------+-----+



In [47]:
# 20. Concatenar las columnas "Nombre" y "Ciudad" en una nueva columna llamada "InformacionPersonal".
from pyspark.sql.functions import concat_ws

df = df.withColumn("InformacionPersonal", concat_ws(", ", df["nombreCompleto"], df["ciudad"]))

df.show()


+--------------+----+-----------+---------+-------------------+
|nombreCompleto|edad|     ciudad|RangoEdad|InformacionPersonal|
+--------------+----+-----------+---------+-------------------+
|         Alice|  25|   New York|    21-40|    Alice, New York|
|           Bob|  30|Los Angeles|    21-40|   Bob, Los Angeles|
|       Charlie|  22|    Chicago|    21-40|   Charlie, Chicago|
+--------------+----+-----------+---------+-------------------+

